# 使用Kernel Anaconda2.7

 <font color=#000000 size=6 face="黑体">  Test model </font><br>
 <font color=#FF00FF size=6 face="黑体"> 1、Imports </font><br>

In [1]:
import numpy as np
import os
import json
import tensorflow as tf
import time
import datetime
import sys
from sklearn.metrics import average_precision_score
import ctypes
from function import *

<font color=#FF00FF size=6 face="黑体"> 2、Get Data & Transform Data </font><br>

In [2]:
#length of sentence
fixlen = 120
#max length of position embedding is 100 (-100~+100)
maxlen = 100

# folder of training datasets
data_path = "./origin_data/"
# files to export data
export_path = "./data/"
word2id = {}
relation2id = {}
word_size = 0
word_vec=None

textual_rel_total, rel_total = init_relation(data_path,export_path,relation2id)
entity_total, word_total,word_vec = init_word(data_path,export_path,word2id, word_size,word_vec)

#print word2id
f = open(data_path + "word2id.txt", "w")
for i in word2id:
	f.write("%s\t%d\n"%(i, word2id[i]))
f.close()
init_kg(data_path,export_path,word2id,relation2id)
np.save(export_path+'vec', word_vec)
f = open(export_path+'config', "w")
f.write(json.dumps({"word2id":word2id,"relation2id":relation2id,"word_size":word_size, "fixlen":fixlen, "maxlen":maxlen, "entity_total":entity_total, "word_total":word_total, "rel_total":rel_total, "textual_rel_total":textual_rel_total}))
f.close()

sort_files(data_path,"test",relation2id, [textual_rel_total, rel_total])
instance_triple, instance_scope, test_len, test_label, test_word, test_pos1, test_pos2, test_mask, test_head, test_tail = init_train_files(data_path,"test_sort",  [textual_rel_total, rel_total],maxlen,fixlen,relation2id,word2id)
np.save(export_path+'test_instance_triple', instance_triple)
np.save(export_path+'test_instance_scope', instance_scope)
np.save(export_path+'test_len', test_len)
np.save(export_path+'test_label', test_label)
np.save(export_path+'test_word', test_word)
np.save(export_path+'test_pos1', test_pos1)
np.save(export_path+'test_pos2', test_pos2)
np.save(export_path+'test_mask', test_mask)
np.save(export_path+'test_head', test_head)
np.save(export_path+'test_tail', test_tail)

print 'read end...'

reading relation ids...
reading word embedding data...
reading test_sort data...
99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999
2099
2199
2299
2399
2499
2599
2699
2799
2899
2999
3099
3199
3299
3399
3499
3599
3699
3799
3899
3999
4099
4199
4299
4399
4499
4599
4699
4799
4899
4999
5099
5199
5299
5399
5499
5599
5699
5799
5899
5999
6099
6199
6299
6399
6499
6599
6699
6799
6899
6999
7099
7199
7299
7399
7499
7599
7699
7799
7899
7999
8099
8199
8299
8399
8499
8599
8699
8799
8899
8999
9099
9199
9299
9399
9499
9599
9699
9799
9899
9999
10099
10199
10299
10399
10499
10599
10699
10799
10899
10999
11099
11199
11299
11399
11499
11599
11699
11799
11899
11999
12099
12199
12299
12399
12499
12599
12699
12799
12899
12999
13099
13199
13299
13399
13499
13599
13699
13799
13899
13999
14099
14199
14299
14399
14499
14599
14699
14799
14899
14999
15099
15199
15299
15399
15499
15599
15699
15799
15899
15999
16099
16199
16299
16399
16499
16599
16699
16799
16899
16999
17099
1719

132099
132199
132299
132399
132499
132599
132699
132799
132899
132999
133099
133199
133299
133399
133499
133599
133699
133799
133899
133999
134099
134199
134299
134399
134499
134599
134699
134799
134899
134999
135099
135199
135299
135399
135499
135599
135699
135799
135899
135999
136099
136199
136299
136399
136499
136599
136699
136799
136899
136999
137099
137199
137299
137399
137499
137599
137699
137799
137899
137999
138099
138199
138299
138399
138499
138599
138699
138799
138899
138999
139099
139199
139299
139399
139499
139599
139699
139799
139899
139999
140099
140199
140299
140399
140499
140599
140699
140799
140899
140999
141099
141199
141299
141399
141499
141599
141699
141799
141899
141999
142099
142199
142299
142399
142499
142599
142699
142799
142899
142999
143099
143199
143299
143399
143499
143599
143699
143799
143899
143999
144099
144199
144299
144399
144499
144599
144699
144799
144899
144999
145099
145199
145299
145399
145499
145599
145699
145799
145899
145999
146099
146199
146299

<font color=#FF00FF size=6 face="黑体"> 3、Modeling </font><br>

In [3]:
os.chdir('jointE') 
import network
export_path = "../data/"

word_vec = np.load(export_path + 'vec.npy')
f = open(export_path + "config", 'r')
config = json.loads(f.read())
f.close()

ll = ctypes.cdll.LoadLibrary   
lib = ll("./init.so")
lib.setInPath("../data/")
lib.init()

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_float('nbatch_kg',100,'entity numbers used each training time')
tf.app.flags.DEFINE_float('margin',1.0,'entity numbers used each training time')
tf.app.flags.DEFINE_float('learning_rate_kg',0.001,'learning rate for kg')
tf.app.flags.DEFINE_float('ent_total',lib.getEntityTotal(),'total of entities')
tf.app.flags.DEFINE_float('rel_total',lib.getRelationTotal(),'total of relations')
tf.app.flags.DEFINE_float('tri_total',lib.getTripleTotal(),'total of triples')
tf.app.flags.DEFINE_float('katt_flag', 1, '1 for katt, 0 for att')

tf.app.flags.DEFINE_string('model', 'cnn', 'neural models to encode sentences')
tf.app.flags.DEFINE_float('max_length',config['fixlen'],'maximum of number of words in one sentence')
tf.app.flags.DEFINE_float('pos_num', config['maxlen'] * 2 + 1,'number of position embedding vectors')
tf.app.flags.DEFINE_float('num_classes', config['textual_rel_total'],'maximum of relations')

tf.app.flags.DEFINE_float('hidden_size',230,'hidden feature size')
tf.app.flags.DEFINE_float('pos_size',5,'position embedding size')

tf.app.flags.DEFINE_float('max_epoch',30,'maximum of training epochs')
tf.app.flags.DEFINE_float('batch_size',131*2,'entity numbers used each training time')
tf.app.flags.DEFINE_float('learning_rate',0.1,'entity numbers used each training time')
tf.app.flags.DEFINE_float('weight_decay',0.00001,'weight_decay')
tf.app.flags.DEFINE_float('keep_prob',1.0,'dropout rate')

tf.app.flags.DEFINE_float('test_batch_size',131*2,'entity numbers used each test time')
tf.app.flags.DEFINE_string('checkpoint_path','./model/','path to store model')

def main(_):

	print 'reading word embedding'
	word_vec = np.load(export_path + 'vec.npy')
	print 'reading test data'
	test_instance_triple = np.load(export_path + 'test_instance_triple.npy')
	test_instance_scope = np.load(export_path + 'test_instance_scope.npy')
	test_len = np.load(export_path + 'test_len.npy')
	test_label = np.load(export_path + 'test_label.npy')
	test_word = np.load(export_path + 'test_word.npy')
	test_pos1 = np.load(export_path + 'test_pos1.npy')
	test_pos2 = np.load(export_path + 'test_pos2.npy')
	test_mask = np.load(export_path + 'test_mask.npy')
	test_head = np.load(export_path + 'test_head.npy')
	test_tail = np.load(export_path + 'test_tail.npy')

	sess = tf.Session()
	if FLAGS.model.lower() == "cnn":
		model = network.CNN(is_training = False, word_embeddings = word_vec)
	elif FLAGS.model.lower() == "pcnn":
		model = network.PCNN(is_training = False, word_embeddings = word_vec)
	elif FLAGS.model.lower() == "lstm":
		model = network.RNN(is_training = False, word_embeddings = word_vec, cell_name = "LSTM", simple_position = True)
	elif FLAGS.model.lower() == "gru":
		model = network.RNN(is_training = False, word_embeddings = word_vec, cell_name = "GRU", simple_position = True)
	elif FLAGS.model.lower() == "bi-lstm" or FLAGS.model.lower() == "bilstm":
		model = network.BiRNN(is_training = False, word_embeddings = word_vec, cell_name = "LSTM", simple_position = True)
	elif FLAGS.model.lower() == "bi-gru" or FLAGS.model.lower() == "bigru":
		model = network.BiRNN(is_training = False, word_embeddings = word_vec, cell_name = "GRU", simple_position = True)
	sess.run(tf.global_variables_initializer())
	saver = tf.train.Saver()

	def test_step(head, tail, word, pos1, pos2, mask, leng, label_index, label, scope):
		feed_dict = {
			model.head_index: head,
			model.tail_index: tail,
			model.word: word,
			model.pos1: pos1,
			model.pos2: pos2,
			model.mask: mask,
			model.len : leng,
			model.label_index: label_index,
			model.label: label,
			model.scope: scope,
			model.keep_prob: FLAGS.keep_prob
		}
		output = sess.run(model.test_output, feed_dict)
		return output
	FLAGS.test_batch_size,FLAGS.num_classes=int(FLAGS.test_batch_size),int(FLAGS.num_classes)
	f = open('results.txt','w')
	f.write('iteration\taverage precision\n')
	for iters in range(1,11):
		print iters
		saver.restore(sess, FLAGS.checkpoint_path + FLAGS.model+str(int(FLAGS.katt_flag))+"-"+str(3664*iters))

		stack_output = []
		stack_label = []
		
		iteration = len(test_instance_scope)/FLAGS.test_batch_size
		for i in range(int(iteration)):
			temp_str= 'running '+str(i)+'/'+str(iteration)+'...'
			sys.stdout.write(temp_str+'\r')
			sys.stdout.flush()
			input_scope = test_instance_scope[i * FLAGS.test_batch_size:(i+1)*FLAGS.test_batch_size]
			index = []
			scope = [0]
			label = []
			for num in input_scope:
				index = index + range(num[0], num[1] + 1)
				label.append(test_label[num[0]])
				scope.append(scope[len(scope)-1] + num[1] - num[0] + 1)
			label_ = np.zeros((FLAGS.test_batch_size, FLAGS.num_classes))
			label_[np.arange(FLAGS.test_batch_size), label] = 1
			output = test_step(test_head[index], test_tail[index], test_word[index,:], test_pos1[index,:], test_pos2[index,:], test_mask[index,:], test_len[index], test_label[index], label_, np.array(scope))
			stack_output.append(output)
			#print 'output',output
			#raise 1
			stack_label.append(label_)
			
        
		print 'evaluating...'

		stack_output = np.concatenate(stack_output, axis=0)
		stack_label = np.concatenate(stack_label, axis = 0)

		exclude_na_flatten_output = stack_output[:,1:]
		exclude_na_flatten_label = stack_label[:,1:]
		average_precision = average_precision_score(exclude_na_flatten_label,exclude_na_flatten_output, average = "micro")

		np.save('./'+FLAGS.model+'+sen_att_all_prob_'+str(iters)+'.npy', exclude_na_flatten_output)
		np.save('./'+FLAGS.model+'+sen_att_all_label_'+str(iters)+'.npy',exclude_na_flatten_label)

		print 'pr: '+str(average_precision)
		f.write(str(average_precision)+'\n')
	f.close()

if __name__ == "__main__":
	tf.app.run()

reading word embedding
reading test data
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

1
INFO:tensorflow:Restoring parameters from ./model/cnn1-3664
evaluating...69...
pr: 0.1026378064962144
2
INFO:tensorflow:Restoring parameters from ./model/cnn1-7328
evaluating...69...
pr: 0.23988183378011432
3
INFO:tensorflow:Restoring parameters from ./model/cnn1-10992
evaluating...69...
pr: 0.2947337408056269
4
INFO:tensorflow:Restoring parameters from ./model/cnn1-14656
evaluating...69...
pr: 0.3396643581221027
5
INFO:tensorflow:Restoring parameters from ./model/cnn1-18320
evaluating...69...
pr: 0.3605053392379348
6
INFO:tensorflow:Restoring parameters from ./model/cnn1-21984
evaluating...69...
pr: 0.3716963856194515
7
INFO:tensorflow:Restoring parameters from ./model/cnn1-25648
ev

SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


<font color=#FF00FF size=6 face="黑体"> 4、Visualisation </font><br>

In [1]:
import os
os.chdir('jointE') 
%run pr_plot.py

auc = 0.21686333417892456 | max F1 = 0.38292184


<img src='jointE/pr.png'  width=500, height=300><br>

<font color=#FF00FF size=6 face="黑体"> 5、Compare with OpenNRE </font><br>

<font color=#000000 size=3 face="黑体">  auc = 0.39101327542801967 | max F1 = 0.4495592556317336  </font><br>
<img src='jointE/pr_curve.png'  width=500, height=300><br>